# Onboard a model defined in a .wf1 file
This notebook takes a .wf1 workfile and transform  it to a modelflow model.<br>

The overall process is performed in a Dataclass named:**GrapWfModel**<br> 
Close study of this class can be rewarding, but is outside the need of most users.<br>
The overall structure is: 
 1. Eviews is started and the wf1 (or wf2) file is loaded
    -    Some transformations are performed on data.
    -    The model is unlinked 
    -    The workspace is saved as a wf2 file. Same name with ```_modelflow appended```
 5. Eviews is closed 
 6. The wf2 file is read as a json file. 
 7. Relevant objects are extracted. 
 7. The MFMSA variable is  extracted, to be saved in the dumpfile. 
 8. The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 9. Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)  
 9. For Stochastic equations new fitted variables are generated - without add add_factors and dummies.  
 9. A model to generate fitted variables is created  
 9. A model to generate add_factors is created. 
 9. A model encompassing the original equations, the model for fitted variables and for add_factors is created. 
 9. The data series and scalars are shoveled into a Pandas dataframe 
     - Some special series are generated as the expression can not be incorporated into modelflow model specifications
     - The model for fitted values is simulated in the specified timespan
     - The model for add_factors is simulated in the timespan set in MFMSA
 10. The data descriptions are extracted into a dictionary. 
    - Data descriptions for dummies, fixed values, fitted values and add_factors are derived. 
 11. Now we have a model and a dataframe with all variables which are needed.
 b
The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  


## Prerequisites  

- Eviews version 12 
- The python library: ```pyevies```  


In [1]:
%matplotlib Notebook

In [2]:
from pathlib import Path

from modelclass import model
from modelgrabwf2 import GrabWfModel
model.widescreen()

## Model specific transformations 
Not all Eviews equations have a direct counterpart in modelflow.<br>
To handle that two types of transformations can be provided for a specific model. 
- **eviews_run_lines** which specifies a list of Eviews command lines to be run. Typical to make some transformation of data which is not possible in modelflow. 
- **country_trans** which specifies a list of replacements to be done to the Eviews formulas before they are processed to modelflow. Typical to use the transformation mentioned above.  

In [10]:
filename = Path('../model/paksolncons.wf1')
modelname = 'pak'

## Process the model

In [12]:
pak_trans = lambda input : input.replace('- 01*D(','-1*D(')                

all_about_pak = GrabWfModel(filename, 
                    country_trans = pak_trans,        
                    make_fitted = False,       # If we want to calculate a fittet value for stocastic eq 
                    do_add_factor_calc=True,   # Stocastic equations are argumented by add factor 
                    start = 2020,
                    end = 2100, 
                    fit_start = 2000,          # Start of calculation of fittet model in baseline 
                    fit_end   = 2030 ) 


Reading M:\modelflow\modelflow-manual\papers\mfbook\content\EViews\..\model\paksolncons.wf1
Assummed model name: PAK


COMError: (-2147221495, 'Not able to get the moniker of the object', ('m:\\modelflow\\modelflow-manual\\papers\\mfbook\\content\\eviews\\..\\model\\paksolncons.wf1 not found on disk in "WFOPEN "M:\\MODELFLOW\\MODELFLOW-MANUAL\\PAPERS\\MFBOOK\\CONTENT\\EVIEWS\\..\\MODEL\\PAKSOLNCONS.WF1"".', 'EViews.Application.13.Run', None, 0, None))

In [ ]:
all_about_pak.model_all_about['mfmsa_options']

## Check if each equation on its own result in the values provided. 
aka: residual check <br> 
If they are not pretty close, something is very wrong. 

In [ ]:
all_about_pak.test_model(all_about_pak.start,
                         all_about_pak.end,
                         maxerr=100,tol=0.00001,
                         showall=0)

## Extract the model and the baseline
**all_about_ken** has a lot of content including. 
- .mmodel is the model instance
- .base_input is the baseline where the add factors and the fitted values are calculated 

In [7]:
#mpak,baseline     = all_about_pak()     
baseline = all_about_pak.base_input
mpak.model_description='Pakistan  model from paksolncons.wf1 27/3-2023'

pak_all_frml_dict = all_about_pak.all_frml_dict


NameError: name 'all_about_pak' is not defined

## Run the model 
In order to achieve numerical stability Gauss-Seidle has to be dampened: alfa=0.7 makes sure that the solution does not explode. 
The convergence criteria is tightend a lot. 

In [ ]:
res = mpak(baseline,2022,2100,silent=1,alfa=1,ldumpvar=0)


## And the simulation result is also fine. 
Here the percent difference is displayed

In [ ]:
mpak['*GGEXPCAPTCN *NYGDPMKTPCN *GGDBTTOTLCN *BNCABFUNDCD PAKNVELECKN']

## Look at a stochastic variable 
Here the equations undergo more phases 

In [ ]:
mpak.model_description

## Now dump the model and data 

In [ ]:
if 1:
    mpak.modeldump('models/270323_paksolncons.pcim')
    if 0: 
        try:
            mpak.modeldump('C:/modelflow manual/model_repo/270323_paksolncons.pcim')
        except: 
            print('could not write to modelrepo file')

In [ ]:
mpakload,baselineload = model.modelload('models/270323_paksolncons.pcim',run = 1,alfa = 0.8)

In [ ]:
xx = mpakload(baseline,2022,2050,alfa=0.7,silent=0)

In [44]:
import fnmatch
reverse_des  = {v:k for k,v in mpak.var_description.items()}
list_des = fnmatch.filter(reverse_des.keys(),'*GDP*')
list_var = [reverse_des[v] for v in list_des]

['PAKNYGDPFCSTKN',
 'PAKNYGDPMKTPCN',
 'PAKNYGDPMKTPKN',
 'PAKNYGDPFCSTXN_A',
 'PAKNYGDPFCSTXN_D',
 'PAKNYGDPFCSTXN_X']

In [37]:
mpak[' '.join( [reverse_des[v] for v in list_des])].frml

PAKNYGDPFCSTKN   : FRML <IDENT> PAKNYGDPFCSTKN = PAKNYGDPMKTPKN-PAKNYTAXNINDKN $
PAKNYGDPMKTPCN   : FRML <IDENT> PAKNYGDPMKTPCN = PAKNECONPRVTCN+PAKNECONGOVTCN+PAKNEGDIFTOTCN+PAKNEGDISTKBCN+PAKNEEXPGNFSCN-PAKNEIMPGNFSCN+PAKNYGDPDISCCN+PAKADAP*PAKDISPREPCN $
PAKNYGDPMKTPKN   : FRML <IDENT> PAKNYGDPMKTPKN = PAKNECONPRVTKN+PAKNECONGOVTKN+PAKNEGDIFTOTKN+PAKNEGDISTKBKN+PAKNEEXPGNFSKN-PAKNEIMPGNFSKN+PAKNYGDPDISCKN+PAKADAP*PAKDISPREPKN $
PAKNYGDPFCSTXN_A : Exogenous
PAKNYGDPFCSTXN_D : Exogenous
PAKNYGDPFCSTXN_X : Exogenous


# Allow for searchable information about model

The below creates a dictionary linking variable names to their eviews variables.

In [5]:
eviews_dict = {v: f.eviews for v,f in all_about_pak.all_frml_dict.items()}
eviews_dict 

NameError: name 'all_about_pak' is not defined

In [50]:

all_dict = {v: {'frml':mpak.allvar[v]['frml'],'des':mpak.var_description[v],
    'eviews':eviews_dict[v]} for v in list_var if v in mpak.endogene}

In [57]:

for v in all_dict.keys():
    print(f'{v:10}  {all_dict[v]["eviews"]}')

PAKNYGDPFCSTKN  @IDENTITY PAKNYGDPFCSTKN  = PAKNYGDPMKTPKN  - PAKNYTAXNINDKN
PAKNYGDPMKTPCN  @IDENTITY PAKNYGDPMKTPCN  = PAKNECONPRVTCN  + PAKNECONGOVTCN  + PAKNEGDIFTOTCN  + PAKNEGDISTKBCN  + PAKNEEXPGNFSCN  - PAKNEIMPGNFSCN  + PAKNYGDPDISCCN  + PAKadap  * PAKDISPREPCN
PAKNYGDPMKTPKN  @IDENTITY PAKNYGDPMKTPKN  = PAKNECONPRVTKN  + PAKNECONGOVTKN  + PAKNEGDIFTOTKN  + PAKNEGDISTKBKN  + PAKNEEXPGNFSKN  - PAKNEIMPGNFSKN  + PAKNYGDPDISCKN  + PAKadap  * PAKDISPREPKN
